In [1]:
#run release to put latest version fbbt.json in /oort

import requests
import json
import pandas as pd
from collections import OrderedDict


In [2]:
#import fbbt from web
#fbbt = requests.get("http://purl.obolibrary.org/obo/fbbt.json").json()
#fbbt['graphs'][0]['edges']

#import fbbt from /oort
fbbt = json.load(open("/Users/clare/git/drosophila-anatomy-developmental-ontology/fbbt.json", "r"))
#fbbt['graphs'][0]['edges']


In [3]:
#find all nervous systems and parts of them neuron = FBbt_00005093, sense organ = FBbt_00005155

terms = ['http://purl.obolibrary.org/obo/FBbt_00005093']
result = ['http://purl.obolibrary.org/obo/FBbt_00005093']
x = 1
while x > 0:
    x = 0
    new_terms = list()
    for e in fbbt['graphs'][0]['edges']:
        if (e['obj'] in terms) and ((e['pred'] == 'http://purl.obolibrary.org/obo/BFO_0000050') \
                                     or (e['pred'] == 'is_a')):
            new_terms.append(e['sub'])
            x += 1
    terms.clear()
    for i in new_terms:
        if i not in result:
            result.append(i)
            terms.append(i)
    new_terms.clear()

#result

In [4]:
len(set(result))

6233

In [6]:
#output list of results - not usually needed
"""
result_df = pd.DataFrame(result)
result_df.to_csv("./result_list.tsv", sep = "\t", header=False, index=False)
"""

'\nresult_df = pd.DataFrame(result)\nresult_df.to_csv("./result_list.tsv", sep = "\t", header=False, index=False)\n'

In [7]:
#generate VFB refs
VFB_dict = OrderedDict([])
for i in result:
    fb_id = i.split("/")[-1]
    VFB_dict[i] = "VFB:" + fb_id

#VFB_dict


In [8]:
# Make a dictionary with key - column header & value = template specification (first row of table).
# Make first two columns

template_seed = OrderedDict([ ('ID' , 'ID'), ('CLASS_TYPE' , 'CLASS_TYPE'),\
                             ('RDF_Type' , 'TYPE' ), ("Xref" , "A oboInOwl:hasDbXref")])

# Create dataFrame for template
# from_records takes a list of dicts - one for each row.  We only have one row.

template = pd.DataFrame.from_records([template_seed])

#template

In [9]:
for i in VFB_dict:

    row_od = OrderedDict([]) #new template row as an empty ordered dictionary
    for c in template.columns: #make columns and blank data for new template row
        row_od.update([(c , "")])
    
    #these are the same in each row
    row_od["CLASS_TYPE"] = "subclass"
    row_od["RDF_Type"] = "owl:Class"
    
    #ID and xref
    row_od['ID'] = i
    row_od["Xref"] = VFB_dict[i]
    
    #make new row into a DataFrame and add it to template
    new_row = pd.DataFrame.from_records([row_od])
    template = pd.concat([template, new_row], ignore_index=True, sort=False)
    
#template

In [10]:
template.to_csv("./template.tsv", sep = "\t", header=True, index=False)